In [23]:
from utils.long_residuals_mlp import *
from utils.utils import *
! pip install --upgrade pip
import cv2
from utils.utils import *
from utils.MLP import *
from utils.long_residuals_mlp import *
from utils.short_residuals_mlp import *

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [18]:
PATCH_SIZE = 32
CLASSES      = ['coast','forest','highway','inside_city','mountain','Opencountry','street','tallbuilding']

MODEL_WEIGHTS = '/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/long_residuals-4096-32-1e-05/long_residuals-4096-32-1e-05_best.h5'

images_directory = '/home/georg/projects/university/C3_ML_for_CV/MIT_split/'


In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Reshape

model = build_long_residuals_mlp(input_shape=(PATCH_SIZE, PATCH_SIZE, 3),
                                  activation='relu', 
                                  regularization_coeff=0.00001, 
                                  num_units=4096, 
                                  filename='myMLP.png')

model.load_weights(MODEL_WEIGHTS)

model_layer = Model(inputs=model.input, outputs=model.get_layer('dense_5').output)


2024-01-14 14:38:39.064640: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 flatten (Flatten)           (None, 3072)                 0         ['input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 4096)                 1258700   ['flatten[0][0]']             
                                                          8                                       
                                                                                                  
 dense_1 (Dense)             (None, 4096)                 1678131   ['dense[0][0]']           

In [19]:
EMBEDDINGS_DIR = '/home/georg/projects/university/C3_ML_for_CV/project23-24-04/Task2/embeddings'
PATCH_SIZE = 32


N_CLUSTERS=128
N_NEIGHBORS = 5
METRIC = 'euclidean'
WEIGHT = 'distance'

In [20]:
embeddings_dict = read_embeddings(directory=os.path.join(EMBEDDINGS_DIR, str(PATCH_SIZE)))

training_files, training_labels = load_data_from_directory('/home/georg/projects/university/C3_ML_for_CV/MIT_split/train')
test_files, test_labels = load_data_from_directory('/home/georg/projects/university/C3_ML_for_CV/MIT_split/test')

training_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in training_files])
test_embeddings = np.array([embeddings_dict[os.path.basename(file)] for file in test_files])

training_embeddings = training_embeddings.reshape((1881 * 64, 256))
test_embeddings = test_embeddings.reshape((807 * 64, 256))

print(training_embeddings.shape)
print(test_embeddings.shape)

(120384, 256)
(51648, 256)


In [21]:
codebook = MiniBatchKMeans(n_clusters=N_CLUSTERS,
                           verbose=False, 
                           batch_size=2150,
                           compute_labels=False,
                           reassignment_ratio=0.01,
                           random_state=42)

codebook.fit(training_embeddings)
# create visual words for training data
train_visual_words = create_visual_words(training_files, embeddings_dict, codebook)
# create visual words for test data
test_visual_words = create_visual_words(test_files, embeddings_dict, codebook)

/home/georg/projects/.venv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


KeyError: '/home/georg/projects/university/C3_ML_for_CV/MIT_split/train/mountain/nat1046.jpg'

In [ ]:
# fit kNN classifier
print(f'Fitting knn with n_neighbors={N_NEIGHBORS}, metric={METRIC}, weights={WEIGHT}')
knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS,n_jobs=-1,metric=METRIC, weights=WEIGHT)
knn.fit(train_visual_words, training_labels)
# predict labels for test data
pred_labels = knn.predict(test_visual_words)

# get accuracy metric for current run
print(get_metrics(test_labels, pred_labels))
